In [1]:
#################################
# filenames
#################################

number = 36

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(11)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(11)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(11)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(11)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(11)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(11)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:101])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615358503.2705932
500
1/500 57/2,b7,57,1
histo1 1
2/500 6,5,27,66
histo1 4
3/500 46,164,4,77
histo1 1
4/500 57,1,4,5/5
histo1 3
5/500 5,1,6,26
histo1 3
6/500 1,M17,665,b27
histo1 1
7/500 6,46,164,5
histo1 2
8/500 67,4,57/6,67
histo1 2
9/500 47,6,56,1
histo1 1
10/500 6,56,6,4
histo1 1
11/500 1,3,5,67
histo1 1
12/500 47,1,5,47
histo1 6
13/500 67,47,27,L77
histo1 1
14/500 6,37,66,6
histo1 3
15/500 664,564,664,2
histo1 1
16/500 6,57/6,6,b6
histo1 1
17/500 564,47,67,4
histo1 1
18/500 5,2,5,6
histo1 7
19/500 77,5/6,6,264
histo1 1
20/500 37,D3,4,b6
histo1 1
21/500 5/6,6,17,1
histo1 1
22/500 27,76,2,27
histo1 1
23/500 5,4,5,L27
histo1 1
24/500 1,57/4,364,6
histo1 1
25/500 1,564,16,57/4
histo1 1
26/500 1,5,1,5/3
histo1 1
27/500 5/6,6,b7,6
histo1 2
28/500 664,2,6,664
histo1 1
29/500 5/6,1,2,5
histo1 1
30/500 2,3,6,565/6
histo1 1
31/500 57/5,665,542/5,543
histo1 1
32/500 5,7/6,6,66
histo1 1
33/500 27,56,1,56
histo1 1
34/500 5,36,6,36
histo1 1
35/500 1,M17,5/5,D47
histo1 1
36/500 164,1,4,b6
histo

histo1 4
291/500 5,5/5,37,L76
histo1 1
292/500 365,27,47,6
histo1 1
293/500 47,L77,7,37
histo1 1
294/500 67,6,76,5/6
histo1 1
295/500 1,b4,1,465
histo1 1
296/500 4,1,5,464
histo1 1
297/500 b6,b7,1,142
histo1 1
298/500 4,57,1,L4
histo1 1
299/500 1,17,57/4,4
histo1 10
300/500 5,1,5,464
histo1 2
301/500 27,4,1,164
histo1 1
302/500 365,4,6,1
histo1 1
303/500 1,17,4,2
histo1 2
304/500 664,564,664,4
histo1 1
305/500 57/6,6,57/6,4
histo1 1
306/500 b7,1,b16,4
histo1 1
307/500 642,47,664,5/2
histo1 1
308/500 464,6,56/6,5
histo1 1
309/500 6,66,37,6
histo1 1
310/500 4,57/6,67,4
histo1 1
311/500 265,6,57/5,57
histo1 1
312/500 642,47,4,6
histo1 1
313/500 1,b4,1,3
histo1 2
314/500 6,5/3,5/6,6
histo1 3
315/500 464,1,464,5/2
histo1 1
316/500 b4,164,5/2,5/5
histo1 1
317/500 5,4,6,7
histo1 1
318/500 6,364,66,664
histo1 1
319/500 56/6,M5,56/6,M5
histo1 1
320/500 7,16,5/6,6
histo1 1
321/500 b4,1,3,4
histo1 3
322/500 1,L27,4,1
histo1 2
323/500 1,57,M17,76/5
histo1 1
324/500 5/6,6,2,66
histo1 1
325/500 1,4,

In [ ]:
##########################
# list errors here
##########################
51/500 6,5/6,6,5
2. Unable to extract for 6 5/6 6 5 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

67/500 4,b7,56/6,264
2. Unable to extract for 4 b7 56/6 264 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [4]:
##########################
# test errors here
##########################
# import requests

# api_url = 'https://api.hooktheory.com/v1/'
# login = {"username": "jl8429",
#          "password": "Hooktheory123"}

# r = requests.post(api_url+'users/auth', data=login)
# #print('r', r)
# #print(r.json())
# '''
# {'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
# '''

# activkey = '0e11e1d65a021d0a883f309f813d74ad'
# header = {"Authorization": "Bearer "+activkey}

mykey = '4,b7,56/6,264'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Too Many Requests","message":"Rate limit exceeded.","code":0,"status":429}'


In [6]:
# ##########################
# # fix histo here
# ##########################
# # with open('histo36.txt', "r") as f:
# #     histo = f.read()
# #     histo = histo[1:-1]
# #     histo = histo.split(', ')
# #     histo = [int(x) for x in histo]

# histo[20] -= 2
# print(histo[:21])
# with open('histo36.txt', "w") as f:  
#     f.write(str(histo))